## **SYS5110 Simulation Project** ##

#### v2.1  ####
Scenario 1: \\
Optimize agent shifts - more agents working during "peak hours", reduced agents working during "low-volume" hours. \\
Achieved by dividing the simulation into time period segments (e.g. 4hr, 8hr, 4hr.) and running the modified simulate_calls() function with the appropriate number of servers each time. \\

\\

Current setup and change from default: \\
Default scenario has 10 agents throughout all 16 hours of work time (6:00am-10:00pm, a.k.a. 0600-2200). \\
New scenario setup as follows: \\
* From 0600-1000, 5 agents working
* From 1000-1800, 15 agents working
* From 1800-2200. 5 agents working

\\
This change corresponds to the call centre allocating work hours for employee shifts by typical demand - a move intended to reduce call abandon rates and waiting times without increasing the overall resources required. \\






## **Code section: Initial parameters & Function definitions** ##


In [846]:
# Basic Parameters
num_queues = 1    # single queue. UNUSED
num_calls = 1000  # Number of calls to simulate
num_servers = 15  # Number of agents (servers) answering calls

## MODIFIED PARAMETERS
num_servers_s1 = 5    # hours 0600-1000
num_servers_s2 = 15   # hours 1000-1800
num_servers_s3 = 5    # hours 1800-2200

num_calls_s1 = 200
num_calls_s2 = 650
num_calls_s3 = 150

starting_time_s1 = 0    #let 0600 be the first minute
starting_time_s2 = 240  # 1000 is 240 minutes after 0600
starting_time_s3 = 720  # 1800 is 720 minutes after 0600

In [847]:
# Generate inter-arrival times for all 1000 calls fitted on distribution
import numpy as np
from scipy.stats import expon

# diurate hourly call distribution (with low-volume and peak times)
def generate_interarrival_times(hourly_calls, minutes_per_hour=60):
    """
    Generate inter-arrival times for a given hourly call volume distribution.

    Parameters:
        hourly_calls: List[int] - Number of calls per hour (6 AM to 10 PM).
        seconds_per_hour: int - Number of seconds in an hour (default: 3600).

    Returns:
        List[float] - Inter-arrival times in seconds for all calls.
    """
    inter_arrival_times = []
    for calls in hourly_calls:
        rate = calls / minutes_per_hour if calls > 0 else 0
        if rate > 0:
            # Generate inter-arrival times for the number of calls in each hour
            times = expon(scale=1 / rate).rvs(size=calls)
            inter_arrival_times.extend(times)
    return inter_arrival_times

# Hourly call preset discrete distribution of 1000 calls over 16 hours of working time
hourly_calls = [50, 50, 50, 50, 50, 50, 87, 87, 88, 88, 100, 100, 50, 50, 25, 25]
data_inter_arrival_times = generate_interarrival_times(hourly_calls)

# Print a few sample inter-arrival times
print("Sample Inter-Arrival Times:", data_inter_arrival_times[:10])
print("Total Number of Inter-Arrival Times Generated:", len(data_inter_arrival_times))


Sample Inter-Arrival Times: [1.1251953848545, 0.8466762489241949, 0.12151881354290085, 0.4676322935063407, 2.5666731768815594, 2.8817952105108953, 1.9517559542655873, 0.29551690488030097, 3.283819813339327, 2.9671378547083926]
Total Number of Inter-Arrival Times Generated: 1000


In [848]:
from scipy.stats import gamma

def generate_waiting_times(num_samples, alpha=1.99, beta=2.79):
    """
    Generate a list of nuum_samples random waiting times based on a gamma distribution.

    Parameters:
    - num_samples (int): Number of waiting times to generate.
    - alpha (float): Shape parameter of the gamma distribution (default is 1.99).
    - beta (float): Scale parameter of the gamma distribution (default is 2.79).

    Returns:
    - list: Randomly generated waiting times.
    """
    return gamma.rvs(a=alpha, scale=beta, size=num_samples).tolist()

# Generate array of waiting times for 1000 calls in the day
data_max_waiting_times = generate_waiting_times(num_calls)
#print(waiting_times)

# Print a few sample max waiting times
print("Sample Max. Waiting Times (in minutes):", data_max_waiting_times[:10])
print("Total Number of Max. Waiting Times Generated:", len(data_max_waiting_times))

Sample Max. Waiting Times (in minutes): [4.045620972321481, 3.6729431842547657, 8.111219913345328, 5.037582886547091, 0.6298920550243977, 5.674306815020151, 1.6352301439302959, 2.2746284179780942, 8.000844250102244, 2.724405564256396]
Total Number of Max. Waiting Times Generated: 1000


In [849]:
# Custom functions for generating service time for one call
def generate_service_time():
    return np.random.exponential(1 / 5) * 30  # Service time ~ Exponential(5)

In [850]:
def simulate_calls(
    num_calls,
    starting_time,
    interarrival_times,
    max_waiting_times,
    generate_service_time,
    num_servers
):
    """
    Simulates call handling in a multi-server queue system.

    Parameters:
        num_calls (int): Number of calls to simulate.
        interarrival_times (list): List of inter-arrival times for each call.
        max_waiting_times (list): List of maximum waiting times for each call before abandonment.
        generate_service_time (function): Function to generate service times for calls.
        num_servers (int): Number of servers available.

    Returns:
        dict: Results of the simulation including arrival times, service times, waiting times,
              departure times, abandoned calls, server utilizations, etc.
    """
    # Initialize variables
    arrival_times = []
    service_times = []
    waiting_times = []
    waiting_times_with_abandon = []
    departure_times = []
    abandon_call_count = 0

    server_utilization = [0] * num_servers
    server_end_times = [0] * num_servers

    next_available_time = -1

    # Simulate calls
    for i in range(num_calls):
        # Generate arrival time; first arrival time set to 0
        if i == 0:
            arrival_time = starting_time
        else:
            interarrival_time = interarrival_times[i]
            arrival_time = arrival_times[-1] + interarrival_time

        # Generate service time (call duration)
        service_time = generate_service_time()

        # Set max. waiting time
        max_waiting_time = max_waiting_times[i]

        # Assign the earliest available server
        next_available_time = min(server_end_times)
        server_index = server_end_times.index(next_available_time)
        if arrival_time >= next_available_time:
            # No waiting, server is free
            waiting_time = 0
            start_service_time = arrival_time
        else:
            # Waiting required
            waiting_time = next_available_time - arrival_time
            # Abandon call if waiting time exceeds maximum waiting time
            if waiting_time > max_waiting_time:
                waiting_times_with_abandon.append(waiting_time)
                abandon_call_count += 1
                continue

            start_service_time = next_available_time

        # Append to lists if call not abandoned
        arrival_times.append(arrival_time)
        service_times.append(service_time)
        waiting_times.append(waiting_time)
        waiting_times_with_abandon.append(waiting_time)

        # Calculate departure time
        departure_time = start_service_time + service_time
        departure_times.append(departure_time)

        # Update server end time and utilization
        server_end_times[server_index] = departure_time
        server_utilization[server_index] += service_time

    # Return simulation results as a dictionary
    return {
        "arrival_times": arrival_times,
        "service_times": service_times,
        "waiting_times": waiting_times,
        "waiting_times_with_abandon": waiting_times_with_abandon,
        "departure_times": departure_times,
        "abandon_call_count": abandon_call_count,
        "server_utilization": server_utilization,
        "next_available_time": next_available_time,
    }


## **Code section: Main program** ##


In [851]:
# Main program
import numpy as np
import pandas as pd
from scipy.stats import expon
import copy


# Initialize variables with pre-existing data:
arrival_times = []  # List to store arrival times
service_times = []  # List to store service times
max_waiting_times = []   # List to store max waiting times before abandonning calls

# Initialize variables with values calculated from simulation
waiting_times = []  # List to store waiting times
waiting_times_with_abandon = []   # List to store waiting times including those who abandonned
waiting_times_abandon_only = []   # List to store waiting times of abandonned calls
departure_times = []  # List to store departure times
answered_call_count = 0 # Number of calls successfully picked up and answered
abandon_call_count = 0  # Number of calls abandoned due to exceeding max. waiting time

abandon_call_count_s1 = 0 # Number of calls abandoned in s1
abandon_call_count_s2 = 0 # Number of calls abandoned in s2
abandon_call_count_s3 = 0 # Number of calls abandoned in s3


# output model measurement (sensitivity analysis) metrics:
server_utilization = [0] * num_servers  # Track total service time handled by each server
server_end_times = [0] * num_servers    # Track when each server will be free; only used for function
### MODIFIED FOR SCENARIO: different server_utilization value for each time period


# set interarrival times of 1000 calls from preset data
interarrival_times = data_inter_arrival_times
# set max. waiting times of 1000 calls from preset data
max_waiting_times = data_max_waiting_times



In [852]:
# Simulate calls for segment 1: 0600-1000; first 200 calls with 5 servers

call_simulation_results1 = simulate_calls(
    num_calls_s1,
    starting_time_s1,
    interarrival_times,
    max_waiting_times,
    generate_service_time,
    num_servers_s1
)


arrival_times = np.append(arrival_times, call_simulation_results1["arrival_times"])
service_times = np.append(service_times, call_simulation_results1["service_times"])
waiting_times = np.append(waiting_times, call_simulation_results1["waiting_times"])
waiting_times_with_abandon = np.append(waiting_times_with_abandon,
                                            call_simulation_results1["waiting_times_with_abandon"])
departure_times = np.append(departure_times, call_simulation_results1["departure_times"])

# Record total simulation time for s1 calls
total_simulation_time_s1 = max(departure_times)

abandon_call_count += call_simulation_results1["abandon_call_count"]
abandon_call_count_s1 += call_simulation_results1["abandon_call_count"]
server_utilization_s1 = call_simulation_results1["server_utilization"]


In [853]:
# Simulate calls for segment 2: 1000-1800; After the first 200 calls, 650 calls with 15 servers

# update starting time from the immediate next_available_time from end of s1 simulation
starting_time_s2 = call_simulation_results1["next_available_time"]

call_simulation_results2 = simulate_calls(
    num_calls_s2,
    starting_time_s2,
    interarrival_times[200:],
    max_waiting_times[200:],
    generate_service_time,
    num_servers_s2
)


arrival_times = np.append(arrival_times, call_simulation_results2["arrival_times"])
service_times = np.append(service_times, call_simulation_results2["service_times"])
waiting_times = np.append(waiting_times, call_simulation_results2["waiting_times"])
waiting_times_with_abandon = np.append(waiting_times_with_abandon,
                                            call_simulation_results2["waiting_times_with_abandon"])
departure_times = np.append(departure_times, call_simulation_results2["departure_times"])

# Record total simulation time for s2 calls
total_simulation_time_s2 = max(departure_times) - total_simulation_time_s1

abandon_call_count += call_simulation_results2["abandon_call_count"]
abandon_call_count_s2 += call_simulation_results2["abandon_call_count"]

server_utilization_s2 = call_simulation_results2["server_utilization"]


In [854]:
# Simulate calls for segment 1: 0600-1000; first 200 calls with 5 servers

# update starting time from the immediate next_available_time from end of s3 simulation
starting_time_s3 = call_simulation_results2["next_available_time"]

call_simulation_results3 = simulate_calls(
    num_calls_s3,
    starting_time_s3,
    interarrival_times[850:],
    max_waiting_times[850:],
    generate_service_time,
    num_servers_s3
)


arrival_times = np.append(arrival_times, call_simulation_results3["arrival_times"])
service_times = np.append(service_times, call_simulation_results3["service_times"])
waiting_times = np.append(waiting_times, call_simulation_results3["waiting_times"])
waiting_times_with_abandon = np.append(waiting_times_with_abandon,
                                            call_simulation_results3["waiting_times_with_abandon"])
departure_times = np.append(departure_times, call_simulation_results3["departure_times"])

# Record total simulation time for s3 calls
total_simulation_time_s3 = max(departure_times) - total_simulation_time_s2

abandon_call_count += call_simulation_results3["abandon_call_count"]
abandon_call_count_s3 += call_simulation_results3["abandon_call_count"]
server_utilization_s3 = call_simulation_results3["server_utilization"]


In [855]:
# Calculate total number of calls answered
answered_call_count = num_calls - abandon_call_count

# Calculate total number of calls answered
answered_call_count_s1 = num_calls_s1 - abandon_call_count_s1

# Calculate total number of calls answered
answered_call_count_s2 = num_calls_s2 - abandon_call_count_s2

# Calculate total number of calls answered
answered_call_count_s3 = num_calls_s3 - abandon_call_count_s3

# Calculate total simulation time
total_simulation_time = max(departure_times)


# Calculate utilization rates for each server in s1
server_utilization_rates_s1 = [
    (util / total_simulation_time_s1) * 100 for util in server_utilization_s1
]

# Calculate utilization rates for each server in s1
server_utilization_rates_s2 = [
    (util / total_simulation_time_s2) * 100 for util in server_utilization_s2
]

# Calculate utilization rates for each server in s1
server_utilization_rates_s3 = [
    (util / total_simulation_time_s3) * 100 for util in server_utilization_s3
]

'''
# Calculate overall utilization rate for each server for all time
sur_s1_weighted = [util * (num_calls_s1 / num_calls) for util in server_utilization_rates_s1]  # weighted 200/1000
sur_s2_weighted = [util * (num_calls_s2 / num_calls) for util in server_utilization_rates_s2]  # 650/1000
sur_s3_weighted = [util * (num_calls_s3 / num_calls) for util in server_utilization_rates_s3]  # 150/1000

avg_server_utilization_rates_overall = (sum(sur_s1_weighted) / num_servers_s1
                                    + sum(sur_s2_weighted) / num_servers_s2
                                    + sum(sur_s3_weighted) / num_servers_s3)
'''

'\n# Calculate overall utilization rate for each server for all time\nsur_s1_weighted = [util * (num_calls_s1 / num_calls) for util in server_utilization_rates_s1]  # weighted 200/1000\nsur_s2_weighted = [util * (num_calls_s2 / num_calls) for util in server_utilization_rates_s2]  # 650/1000\nsur_s3_weighted = [util * (num_calls_s3 / num_calls) for util in server_utilization_rates_s3]  # 150/1000\n\navg_server_utilization_rates_overall = (sum(sur_s1_weighted) / num_servers_s1 \n                                    + sum(sur_s2_weighted) / num_servers_s2 \n                                    + sum(sur_s3_weighted) / num_servers_s3)\n'

## **Code section: Output & Model metrics** ##

#### *Metrics used:* ####

Average waiting time (including abandoned calls)

Average waiting time (excluding abandoned calls)

Average server utilization

Abandoned calls

Abandoned calls rate

Graph distribution for abandoned calls throughout the day(?)




In [856]:

# Create a DataFrame to display results
df = pd.DataFrame({
    #"Call": range(1, num_calls + 1),
    "Call": range(1, answered_call_count + 1),
    "Arrival Time (min)": arrival_times,
    "Waiting Time (min)": waiting_times,
    "Service Time (min)": service_times,
    "Departure Time (min)": departure_times,
})

# Display the results
print("Call Data:")
print(df)
#print(df.head(20))

# Display server utilization rates
print("\nS1 Server Utilization Rates:")
for i, rate in enumerate(server_utilization_rates_s1):
    print(f"Server {i + 1}: {rate:.2f}%")

print("\nS2 Server Utilization Rates:")
for i, rate in enumerate(server_utilization_rates_s2):
    print(f"Server {i + 1}: {rate:.2f}%")

print("\nS3 Server Utilization Rates:")
for i, rate in enumerate(server_utilization_rates_s3):
    print(f"Server {i + 1}: {rate:.2f}%")




Call Data:
     Call  Arrival Time (min)  Waiting Time (min)  Service Time (min)  \
0       1            0.000000                 0.0            0.503801   
1       2            0.846676                 0.0            3.988155   
2       3            0.968195                 0.0            0.278633   
3       4            1.435827                 0.0            4.768866   
4       5            4.002501                 0.0            4.198201   
..    ...                 ...                 ...                 ...   
847   848          768.349337                 0.0            2.959931   
848   849          768.468145                 0.0            0.586280   
849   850          772.212255                 0.0            9.442056   
850   851          773.552860                 0.0            1.916792   
851   852          774.409392                 0.0            3.892441   

     Departure Time (min)  
0                0.503801  
1                4.834831  
2                1.246828  


In [857]:
# Metrics Calculation

# Average waiting time (including abandoned calls)
average_waiting_time_including_abandon = (
    sum(waiting_times_with_abandon) / len(waiting_times_with_abandon)
    if waiting_times_with_abandon.size > 0 else 0
)

# Average waiting time (excluding abandoned calls)
average_waiting_time_excluding_abandon = (
    sum(waiting_times) / len(waiting_times) if waiting_times.size > 0 else 0
)

# Average server utilization
average_server_utilization_s1 = sum(server_utilization_rates_s1) / num_servers_s1
average_server_utilization_s2 = sum(server_utilization_rates_s2) / num_servers_s2
average_server_utilization_s3 = sum(server_utilization_rates_s3) / num_servers_s3
average_server_utilization = (average_server_utilization_s1 * num_calls_s1 / num_calls
                              + average_server_utilization_s2 * num_calls_s2 / num_calls
                              + average_server_utilization_s3 * num_calls_s3 / num_calls)

# Abandoned calls
abandoned_calls = abandon_call_count

# Abandoned calls rate
abandoned_calls_rate = (abandon_call_count / num_calls) * 100

# Abandoned calls
abandoned_calls_s1 = abandon_call_count_s1
# Abandoned calls
abandoned_calls_s2 = abandon_call_count_s2
# Abandoned calls
abandoned_calls_s3 = abandon_call_count_s3

# Abandoned calls rate
abandoned_calls_rate_s1 = (abandon_call_count_s1 / num_calls_s1) * 100
# Abandoned calls rate
abandoned_calls_rate_s2 = (abandon_call_count_s2 / num_calls_s2) * 100
# Abandoned calls rate
abandoned_calls_rate_s3 = (abandon_call_count_s3 / num_calls_s3) * 100

# Display Metrics
print("Simulation Metrics:")
print(f"1. Average Waiting Time (Including Abandoned Calls): {average_waiting_time_including_abandon:.2f} minutes")
print(f"2. Average Waiting Time (Excluding Abandoned Calls): {average_waiting_time_excluding_abandon:.2f} minutes")

print(f"3. Time-Average Server Utilization: {average_server_utilization:.2f}%")
print(f"  3a. Average Server Utilization from 0600-1000: {average_server_utilization_s1:.2f}%")
print(f"  3a. Average Server Utilization from 1000-1800: {average_server_utilization_s2:.2f}%")
print(f"  3a. Average Server Utilization from 1800-2200: {average_server_utilization_s3:.2f}%")

print(f"4. Number of Abandoned Calls: {abandoned_calls}")
print(f"  4a. Number of Abandoned Calls from 0600-1000: {abandoned_calls_s1}")
print(f"  4b. Number of Abandoned Calls from 1000-1800: {abandoned_calls_s2}")
print(f"  4c. Number of Abandoned Calls from 1800-2200: {abandoned_calls_s3}")
print(f"5. Abandoned Calls Rate: {abandoned_calls_rate:.2f}%")
print(f"  5a. Abandoned Calls Rate from 0600-1000: {abandoned_calls_rate_s1:.2f}%")
print(f"  5b. Abandoned Calls Rate from 1000-1800: {abandoned_calls_rate_s2:.2f}%")
print(f"  5c. Abandoned Calls Rate from 1800-2200: {abandoned_calls_rate_s3:.2f}%")



Simulation Metrics:
1. Average Waiting Time (Including Abandoned Calls): 1.54 minutes
2. Average Waiting Time (Excluding Abandoned Calls): 0.38 minutes
3. Time-Average Server Utilization: 56.66%
  3a. Average Server Utilization from 0600-1000: 78.82%
  3a. Average Server Utilization from 1000-1800: 52.71%
  3a. Average Server Utilization from 1800-2200: 44.20%
4. Number of Abandoned Calls: 148
  4a. Number of Abandoned Calls from 0600-1000: 124
  4b. Number of Abandoned Calls from 1000-1800: 0
  4c. Number of Abandoned Calls from 1800-2200: 24
5. Abandoned Calls Rate: 14.80%
  5a. Abandoned Calls Rate from 0600-1000: 62.00%
  5b. Abandoned Calls Rate from 1000-1800: 0.00%
  5c. Abandoned Calls Rate from 1800-2200: 16.00%


In [858]:
######

### Usage ###

You can change the default 10* coefficient in the generate_service_time() function to scale the service time length distribution. (e.g. multiply by 10 to increase the average service time length by 10 times.) Also possible to experiment with other distributions for varying results. \\
If num_calls is changed (from default 1000), must update hourly_calls array distribution such that the sum matches num_calls (or greater). \\



-------------------------------------------

### Scenarios TODO: ###

1. Additional agent(s)/server for entire duration \\
2. Improved agency working efficiency, i.e. lower service time \\
3. Efficient resource allocation ---> **THIS**. Complete \\

### Output TODO: ###

Graphs requested: \\
  Interarrival time distribution graph (Gamma CDF) \\
  Service time distribution graph \\

  Waiting time scatterplot for all callers \\


